## 1. Imports & setup
## 2. Load trained model
## 3. Load APTOS test dataloader
## 4. Run inference (NO training)
## 5. Metrics computation
## 6. Confusion Matrix
## 7. ROC Curve
## 8. (Optional) Training curves
## 9. Clinical interpretation (Markdown)

# Phase 2 Results Notebook
# Binary DR Screening Model (EyePACS → APTOS)

In [1]:
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data.datamodule import FundusDataModule
from src.models.cnn_backbone import create_backbone
from src.training.evaluate import BinaryEvaluator

plt.style.use("seaborn-v0_8")


In [ ]:
# Load training history
# FIXED: Using raw string to avoid escape sequence issues
history_path = r"D:\TAB\Mine\College\SEM 7\GR\eye-realtime-inference\models\binary_dr\training_history.json"

with open(history_path, "r") as f:
    history = json.load(f)

history.keys()


In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
# Recreate model architecture (same as Phase 2B training)
# FIXED: Corrected parameter name and added missing closing parenthesis
from src.models.cnn_backbone import create_backbone

model = create_backbone(
    backbone="resnet18",
    pretrained=False   # IMPORTANT: must be False when loading trained weights
)

model = model.to(device)


In [ ]:
# Load trained model weights
checkpoint_path = r"D:\TAB\Mine\College\SEM 7\GR\eye-realtime-inference\models\binary_dr\best_model.pth"

checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded model from epoch {checkpoint.get('epoch', 'unknown')}")
print(f"Best validation metrics: {checkpoint.get('metrics', {})}")


In [ ]:
# Load APTOS test dataloader
from src.data.datamodule import FundusDataModule

# Initialize datamodule with APTOS dataset
dm = FundusDataModule(
    data_dir=r"D:\TAB\Mine\College\SEM 7\GR\eye-realtime-inference\data",
    dataset_name="aptos",
    batch_size=32,
    num_workers=4,
    task="binary"
)

dm.setup(stage="test")
aptos_loader = dm.test_dataloader()

print(f"APTOS test samples: {len(aptos_loader.dataset)}")


# CONFUSION MATRIX (APTOS — EXTERNAL TEST)

In [ ]:
# CONFUSION MATRIX (APTOS — EXTERNAL TEST)
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

model.eval()

y_true = []
y_pred = []
y_prob = []

with torch.no_grad():
    for images, labels in aptos_loader:
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        probs = torch.sigmoid(logits).squeeze()

        preds = (probs >= 0.5).int()

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)


# Cell: Confusion Matrix Plot

In [ ]:
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["NORMAL", "DR"]
)

plt.figure(figsize=(6, 6))
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix — APTOS (External Test)")
plt.show()
